# Lab 3


In [86]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
import numpy as np

In [143]:
# variables
OFFSPRING_SIZE = 200
K= 5
POPULATION_SIZE = 50
TOURNAMENT_SIZE = 5


# The Nim and Nimply classes

In [88]:
Nimply = namedtuple("Nimply", "row, num_objects") # move

In [89]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

# Agents


In [90]:
#Agent playing with nim_sum
def nim_sum_l(nim):
  X = nim._rows[0]
  # X is the nim_sum(bitwise xor) of all heap sizes
  for i in range(1, len(nim._rows)):
    X = X ^ nim._rows[i]

  nim_sum_val = []
  # calculate the nim_sum between X and each heap size
  for i in nim._rows:
    val = i ^ X
    nim_sum_val.append(val)

  row = "false"
 
  for i in range(len(nim_sum_val)):
    if nim_sum_val[i] < nim._rows[i]:
      row = i
      break
  
  # reduce that heap to value nim_sum
  if (row != "false"):
    num_objects = nim._rows[row] - nim_sum_val[row]
    move = Nimply(row, num_objects)
  
  else:
    rand_row = random.randrange(0,len(nim._rows))

    while(nim._rows[rand_row] == 0):
      rand_row = random.randrange(0,len(nim._rows))

    if(nim._rows[rand_row] != 1):
      rand_obj = random.randrange(1, nim._rows[rand_row])
                                  
    else: rand_obj = 1
    move = Nimply(rand_row, rand_obj)
  return move

# Evolved Strategy

0 the middle column
pick one from the longest
pick one from random rows
zero the smallest row
half the biggest row
take k sticks if possible otherwise take the max <br />

onerow-stategy: only one row - leave x
2 rows: 1 rad = 1 element. ta x element från den andra raden
2 rows: both has more than 1 element - take random
else: take random random. 

genome: dict 
individ = {rule1: x, rule2:y, rule3:z, rule4:n}

In [91]:
#Remove x element from the only row left
# 0 < x <= k
def one_row_strategy(state: Nim, individual) -> Nimply:
    row_counter = -1
    for row in state.rows: 
        row_counter += 1
        if row > 0:
            return Nimply(row_counter , min(row, individual[0]))


In [92]:
def zero_middle_column(nim):
    #find the row with the amount of sticks that are in the middle
    sorted_rows = sorted(nim._rows)

    middle_row = int(len(sorted_rows)/2) #make sure it is an even number

    obj = sorted_rows[middle_row]

    for i in nim._rows:
        if nim._rows[i] == obj:
            final_row = i

    move = Nimply(i, obj)
    
    return move


# MinMax


In [93]:
def all_possible_moves(nim):
    possible_moves = []

    for row in range(len(nim._rows)):
        for j in range(nim._rows[row]):
            copy_row = deepcopy(row)

            new_move = copy_row[row] - j - 1
            possible_moves.append(new_move)

    return possible_moves
            

In [94]:
def evaluate_nim(nim, player):
    if not nim: # if game is finished
        # player 0 loose if return -1
        return -1 if player == 0 else 1


# Sample strategies

In [95]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [96]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result

In [97]:
def active_rows_index(state: Nim) -> list:
    active_rows_index = []
    count = 0
    for o in state.rows:
        if o > 0:
          active_rows_index.append(count)
        count += 1
    return active_rows_index

In [98]:
def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["active_rows_index"] = active_rows_index(state)
    cooked["rows_with_one_element"] = [(index, r) for index, r in enumerate(state.rows) if r == 1]
    
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)
    
    cooked["pure_random"] = pure_random(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [99]:
def optimal_startegy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [135]:
#An agent returning a random move
def random_agent(state: Nim):
    data = cook_status(state)
    return data["pure_random"]
    

In [101]:
#A dumb agent that always picks one element from the longest row
def dumb(state: Nim):
    data = cook_status(state=state)
    row = data["longest_row"]
    return Nimply(row, 1)

In [102]:
#Create the agents that will play as opponents:
# One that plays with the nim_sum strategy
# One that plays using a random strategy
# One that always pick one element from the longest row

OPPONENT = [nim_sum_l, rand, dumb]

# Rules

In [103]:
def rule_two(state: Nim, data:dict, genome: dict):
  active_rows_index = data["active_rows_index"]
  single_elem_rows = data["rows_with_one_element"]

  if genome["Rule2"][0] == 0:  # take from row with one elem
    elem = 1 # want to take the last elem in row
    single_elem_row = single_elem_rows[0][0] # looks like [(row,elem),(row,elem)] if two rows with single elem
    ply = Nimply(single_elem_row, 1)

  else: #take from the row with more than one element
    if len(single_elem_rows) > 1:
      row = single_elem_rows[0][0]
    else:
      row = [i for i in active_rows_index if state.rows[i] != 1][0]
    
    if (genome['Rule2'][1] > row): # if it wants to leave more elem than exists in row
      elem = 1
      ply = Nimply(row, 1)
                      
    else: 
      elem = max(state.rows[row] - genome['Rule2'][1], 1)
      ply = Nimply(row, elem)
      
  return ply

In [104]:
def rule_three(state: Nim, data: dict, genome: dict):
  #logging.debug('Coosing rule 3')
  if (genome['Rule3'][0] == 0): # choose from row with fewest elemt
    row = data['shortest_row']
  else:
    row = data['longest_row']
      
  elem = max(state.rows[row] - genome['Rule3'][1], 1)
  ply = Nimply(row, elem)

  return ply

In [105]:
def two_rows_left(state: Nim, data: dict, genome: dict ) -> Nimply:
  
  rows_with_one_elem = data["rows_with_one_element"]
 
  if rows_with_one_elem: # Rule 2
    #logging.debug("Choosing rule 2")
    ply = rule_two(state=state, data=data, genome=genome)
  
  else: # Rule 3              
    ply = rule_three(state, data, genome)

  return ply

In [106]:
"""
Rule 1
"""
def one_row_left(state: Nim, data: dict, genome: dict) -> Nimply:
    
    #Rule 1 - if only one row left: leave x sticks
    active_row = data["active_rows_index"][0]
    elem_last_row = state.rows[active_row] #active_rows_index returns a list -> need to get the first one
   
    if elem_last_row < genome['Rule1']: #if the rule want to leave more sticks than exists in row
        ply = Nimply(active_row, 1)
        
    else:
        #add if you want to take more than K - take k
        #if (k <)
        
        elem_to_remove = max(elem_last_row - genome["Rule1"], 1) #if take more elem than exists -> take all
        ply = Nimply(active_row, elem_to_remove)

    return ply
  

In [107]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        active_rows_number = data["active_rows_number"]

        #only one active row left
        if active_rows_number == 1:
            #logging.debug('Choosing rule 1')
            ply = one_row_left(state=state,data=data, genome=genome) 
        
        elif active_rows_number == 2:
           ply = two_rows_left(state=state, data=data, genome=genome)
      
        else:
            move = random.choice(data["possible_moves"])
            ply = Nimply(move[0], move[1])

        return ply

    return evolvable

In [108]:
nim = Nim(3)

ply = Nimply(2,5)
nim.nimming(ply)
ply = Nimply(1,3)
nim.nimming(ply)

data = cook_status(nim)

print(nim)

genome = {'Rule1': 1, 'Rule2': [0,3], 'Rule3': [1,10]}

pl = make_strategy(genome=genome)
ply = pl(nim)
print(ply)
nim.nimming(ply)

#print("rows_with_one_elem", data["rows_with_one_element"])

#print("rdfjh",state.rows[data["active_rows_index"][0]])

#one_match = [(index, r) for index, r in enumerate(state.rows) if r == 1]

<1 0 0>
Nimply(row=0, num_objects=1)


In [109]:
from tqdm import tqdm
#compute the fittness
def head2head(individual1: dict, opponent: Callable, nim_size: int):
    won = 0
    NUM_MATCHES = 1 
    nim = Nim(nim_size)

    players = (make_strategy(individual1), opponent)
    player = 0
    
    #logging.debug(f"status: Initial board  -> {nim}")
    while nim:
        ply = players[player](nim)
        nim.nimming(ply)
        #logging.debug(f"status: After player {player} -> {nim}")
        player = 1 - player
    winner = 1 - player
    
    if winner == 0:
        individual1["fitness"] +=1
        #logging.debug('Player won over agent')
        #öka fitness på individ1 vid vinst
   
    
   


In [110]:
indv1 = {'Rule1': 2, 'Rule2': [1,3], 'Rule3': [1,10], 'fitness':0}
indv2 = {'Rule1': 0, 'Rule2': [1,2], 'Rule3': [2,2], 'fitness':0}
#OPPONENT = [nim_sum_l, rand, dumb]

#opponent = OPPONENT[2]

#head2head(indv1, opponent, 3)

print(indv1)




{'Rule1': 2, 'Rule2': [1, 3], 'Rule3': [1, 10], 'fitness': 0}


In [111]:
# calculate fitness for whole population
def calculate_fitness(population: list, nim_size):
    NUM_MATCHES = 10
    for strat in OPPONENT:
        for p1 in population:
            for _ in range(NUM_MATCHES):
                head2head(individual1=p1, opponent=strat, nim_size=nim_size)


In [112]:
"""
def evaluate(player1: dict, player2: dict, nim_size: int):
    # play 10 times against each other
    games = 10
    player_one_first = True
    while games > 0:
        if player_one_first:
            head2head(player1, player2, nim_size)
        else:
            head2head(player2, player1, nim_size)
        games -=1
"""

'\ndef evaluate(player1: dict, player2: dict, nim_size: int):\n    # play 10 times against each other\n    games = 10\n    player_one_first = True\n    while games > 0:\n        if player_one_first:\n            head2head(player1, player2, nim_size)\n        else:\n            head2head(player2, player1, nim_size)\n        games -=1\n'

In [113]:

def init_population(nim_size: int) -> list:
    population = []
    max_leave = (nim_size-1)*2 # last row of the table will have nim_size*2-1 objects

    cond = POPULATION_SIZE
    
    while cond:
        individual = {'Rule1': random.randint(0,max_leave), 'Rule2': [random.randint(0,1), max_leave], 'Rule3': [random.randint(0, 1), random.randint(0, max_leave)]}
        individual['fitness'] = 0
        population.append(individual)
        cond -= 1
    return population

In [114]:
def k_fittest_indv(population: list, k: int) -> list:
    return sorted(population, key=lambda indv: indv['fitness'], reverse=True)[:k]


In [115]:
def clean_fitness(individuals: list):
    for indv in individuals:
        indv['fitness'] = 0

In [116]:
# select k random individuals, return the one with best fitness
def tournament(population: list, nim_size: int) -> dict:    
    contestors = random.sample(population, TOURNAMENT_SIZE)
    best_contestor = sorted(contestors, key=lambda indv: indv['fitness'], reverse=True)[0]

    #clean_fitness(contestors)

    #for p1 in contestors:
    #    for p2 in contestors:
    #        if p1!=p2:
    #            head2head(individual1=p1, opponent=opponent, nim_size=nim_size)
    
    #best_contestor = sorted(contestors, key=lambda indv: indv['fitness'], reverse=True)[0]
  
    return best_contestor
    

In [117]:
# take two parents and create one child
def crossover(parent1: dict, parent2: dict, mutation_prob: float) -> dict:
    rules_parent1 = [key for key in parent1.keys() if 'Rule' in key]
    
    child = {}

    for rule in rules_parent1:
        which_parent = random.randint(0,1)
        if which_parent == 0: # take rule from parent1
            child[rule] = parent1[rule]
        else:
            child[rule] = parent2[rule]
    
    child['fitness'] = 0
    rules = [key for key in child.keys() if 'Rule' in key]

    #mutation
    if random.random() < mutation_prob:
        rule = random.choice(rules)
        r1 = parent1[rule]
        r2 = parent2[rule]

        if rule == 'Rule1':
            mean = int((r1+r2)/2)
            child[rule] = mean

        else:
            mean_val_one = int((r1[0]+r2[0])/2)
            mean_val_two = int((r1[1]+r2[1])/2)
            child[rule] = [mean_val_one, mean_val_two]
    #logging.INFO('Child: {child}')  
    return child

     

In [118]:
indv1 = {'Rule1': 2, 'Rule2': [0,3], 'Rule3': [1,10], 'fitness':0}
indv2 = {'Rule1': 0, 'Rule2': [1,2], 'Rule3': [2,2], 'fitness':0}

crossover(indv1, indv2, 0.8)

{'Rule1': 1, 'Rule2': [0, 3], 'Rule3': [2, 2], 'fitness': 0}

# Evolution strategy described

1. Create inital population (with POPULATION_SIZE) where each player has the same set of rules but different parameters
2. Generate offspring where OFFSPRING_SIZE>>POPULATION_SIZE:
    - k individuals compete against each other and the fittest becomes a parent
    - Preform crossover between two parents and create one child
3. Play all children against each other to calculate fitness
4. Pick out the top l 
2. Evaluate:
    - Calculate fitness - all children plays agains each other (winner increase fitness)
3. Select the top k fittest children
4. Repeat 2-3 steps GENERATION amount of times


In [119]:
def create_offspring(population: list, nim_size: int, mutation_prob: float) -> list:
    offspring = []
    for i in range(OFFSPRING_SIZE):
        parent1 = tournament(population=population, nim_size=nim_size)
        parent2 = tournament(population=population, nim_size=nim_size)

        child = crossover(parent1, parent2, mutation_prob=mutation_prob)
        
        offspring.append(child)
    return offspring

In [120]:
def get_next_generation(population: list) -> list:
    best_k_offspring = sorted(population, key=lambda child: child['fitness'], reverse=True)[:POPULATION_SIZE]

    return best_k_offspring


In [132]:
# GAME
nim_size = 5
GENERATIONS = 2

def evolution_agent(nim_size: int):
    population = init_population(nim_size=nim_size) 
    #logging.debug(f"Initial pop {population}")
    
    for _ in range(GENERATIONS):
        #print(offspring)
        #evaluate(offspring[0], data[""], nim_size)
        offspring = create_offspring(population, nim_size, 0.2)
        #logging.debug(f"Created Offspring {offspring}")
        calculate_fitness(population=offspring, nim_size=nim_size)
        #logging.debug(f"Calculated fitness {offspring}")
        #offspring = create_offspring(population=next_gen, nim_size=nim_size, mutation_prob=0.5)
        population = get_next_generation(population=offspring)
        
    best_indv = population[0]
    logging.debug(f"best player {best_indv}")
    return best_indv


In [133]:


best = evolution_agent(3)


print(best)

DEBUG:root:best player {'Rule1': 0, 'Rule2': [1, 4], 'Rule3': [1, 4], 'fitness': 16}


{'Rule1': 0, 'Rule2': [1, 4], 'Rule3': [1, 4], 'fitness': 16}


Oversimplified match

In [147]:
logging.getLogger().setLevel(logging.DEBUG)


def play_nim(agent1: Callable, opponent: Callable):
    nim = Nim(5)
    logging.debug(f"status: Initial board  -> {nim}")
    strategy = (agent1, opponent)
    player = 0

    while nim:
        ply = strategy[player](nim)
        nim.nimming(ply)
        logging.debug(f"status: After player {player} -> {nim}")
        player = 1 - player
    winner = 1 - player
    logging.info(f"status: Player {winner} won!")

play_nim(make_strategy(evolution_agent(5)), dumb)



DEBUG:root:best player {'Rule1': 0, 'Rule2': [0, 8], 'Rule3': [1, 7], 'fitness': 19}
DEBUG:root:status: Initial board  -> <1 3 5 7 9>
DEBUG:root:status: After player 0 -> <0 3 5 7 9>
DEBUG:root:status: After player 1 -> <0 3 5 7 8>
DEBUG:root:status: After player 0 -> <0 3 5 3 8>
DEBUG:root:status: After player 1 -> <0 3 5 3 7>
DEBUG:root:status: After player 0 -> <0 3 4 3 7>
DEBUG:root:status: After player 1 -> <0 3 4 3 6>
DEBUG:root:status: After player 0 -> <0 0 4 3 6>
DEBUG:root:status: After player 1 -> <0 0 4 3 5>
DEBUG:root:status: After player 0 -> <0 0 4 2 5>
DEBUG:root:status: After player 1 -> <0 0 4 2 4>
DEBUG:root:status: After player 0 -> <0 0 0 2 4>
DEBUG:root:status: After player 1 -> <0 0 0 2 3>
DEBUG:root:status: After player 0 -> <0 0 0 2 2>
DEBUG:root:status: After player 1 -> <0 0 0 1 2>
DEBUG:root:status: After player 0 -> <0 0 0 0 2>
DEBUG:root:status: After player 1 -> <0 0 0 0 1>
DEBUG:root:status: After player 0 -> <0 0 0 0 0>
INFO:root:status: Player 0 won!
